In [4]:
%pip install -r tic-tac-toe/requirements.txt
%pip install -e tic-tac-toe/

Note: you may need to restart the kernel to use updated packages.
Obtaining file:///C:/Users/Spencer/Documents/CUBoulder/CSCI7000/Assignment1/tic-tac-toe
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for tictactoe (pyproject.toml): started
  Building editable for tictactoe (pyproject.toml): finished with status 'done'
  Created wheel for tictactoe: filename=tictactoe-0.0.1-0.editable-py3-none-any.whl size=2825 sha256=cbdebc061e4d53f4ba293e67163b676181f7936a2eaf2600b31d1b0a4d7d4332
  Stored in directory: C:\Users\Spencer\AppD

In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import json
import numpy as np
from tictactoe.board import Board

import torch
from torch import nn

# Tasks
- Create Data
    - Create Recursive function to get all board positions
    - Create Function to get best move
- Create Transformer
- Train Transformer
- Write article

## Data Work

### Generate Data

In [357]:
size = 3
boards = {}
def generate_data(board):
    b = Board()
    if np.array2string(board.flatten()) not in list(boards.keys()):
        out_boards = []
        for i,j in b.get_best_moves(board):
            nb = Board()
            nb.set_board(np.copy(board))
            nb.place(i,j)
            out_boards.append(nb.to_str())
        boards[np.array2string(board.flatten())] = out_boards

    for i, j in list(zip(*np.where(board==0))):
        nb = Board()
        nb.set_board(np.copy(board))
        nb.place(i,j)
        new_board = nb.board
        try:
            generate_data(new_board)
        except:
            print(f"i: {i} j: {j} board:{new_board}")
            raise
generate_data(np.array([[0,0,0],
                        [0,0,0],
                        [0,0,0]]))

i: 1 j: 0 board:[[ 0  1  1]
 [-1 -1  0]
 [-1 -1  1]]
i: 2 j: 2 board:[[ 0  1  1]
 [ 0 -1  0]
 [-1 -1  1]]
i: 1 j: 1 board:[[ 0  1  1]
 [ 0 -1  0]
 [-1 -1  0]]
i: 0 j: 1 board:[[ 0  1  1]
 [ 0  0  0]
 [-1 -1  0]]
i: 2 j: 0 board:[[ 0  0  1]
 [ 0  0  0]
 [-1 -1  0]]
i: 0 j: 2 board:[[ 0  0  1]
 [ 0  0  0]
 [ 0 -1  0]]
i: 2 j: 1 board:[[ 0  0  0]
 [ 0  0  0]
 [ 0 -1  0]]


KeyboardInterrupt: 

In [370]:
fixed_data = {k:v for k, v in data.items() if v != []}

#### Save Data

In [4]:
len(list(boards.keys()))

6046

In [5]:
ex_key = list(boards.keys())[0]
print(f"ex_key: {ex_key} \n ex_vals: {boards[ex_key]}")

ex_key: [0 0 0 0 0 0 0 0 0] 
 ex_vals: ['[-1  0  0  0  0  0  0  0  0]', '[ 0 -1  0  0  0  0  0  0  0]', '[ 0  0 -1  0  0  0  0  0  0]', '[ 0  0  0 -1  0  0  0  0  0]', '[ 0  0  0  0 -1  0  0  0  0]', '[ 0  0  0  0  0 -1  0  0  0]', '[ 0  0  0  0  0  0 -1  0  0]', '[ 0  0  0  0  0  0  0 -1  0]', '[ 0  0  0  0  0  0  0  0 -1]']


In [371]:
import json
with open('./data/data.json', 'w') as f:
    json.dump(fixed_data, f, indent=4)

#### Load Data

In [7]:
import json

with open('./data/data.json', 'r') as f:
    data = json.load(f)

In [8]:
len(list(data.keys()))

5920

#### Split the Data

In [374]:
import random

train_percent, val_percent, test_percent = 0.7, 0.2, 0.1

data_keys = list(data.keys())
n = len(data_keys)
num_train, num_val, num_test = int(train_percent*n), int(val_percent*n), int(test_percent*n)

select_train = random.sample(data_keys, num_train)
val_and_test = [value for value in data_keys if value not in select_train]

select_val = random.sample(val_and_test, num_val)
select_test = [value for value in val_and_test if value not in select_val]

In [375]:
def boards_to_format(key):
    key = key[1:-1]
    return [int(c) for c in key.split(' ') if c in ['-1', '0', '1']]

train_data = {int(i): (boards_to_format(k), [boards_to_format(v) for v in data[k]]) for i, k in enumerate(select_train)}
val_data = {int(i): (boards_to_format(k), [boards_to_format(v) for v in data[k]]) for i, k in enumerate(select_val)}
test_data = {int(i): (boards_to_format(k), [boards_to_format(v) for v in data[k]]) for i, k in enumerate(select_test)}

In [376]:
print(len(list(train_data.keys())))
print(len(list(val_data.keys())))
print(len(list(test_data.keys())))

4144
1184
592


#### Save the Data

In [377]:
with open('./data/train.json', 'w') as f:
    json.dump(train_data, f)
with open('./data/val.json', 'w') as f:
    json.dump(val_data, f)
with open('./data/test.json', 'w') as f:
    json.dump(test_data, f)

### Translate Data into dataloaders

In [6]:
from torch.utils.data import Dataset
class TicTacDataset(Dataset):
    def __init__(self, path = './data/train.json'):
        with open(path, 'r') as f:
            self.data = json.load(f)
        self.len = len(list(self.data.keys()))
        
        # self.inputs, self.outputs = [], []
        # for ins, outs in data.items():
        #     for out in outs:
        #         self.inputs.append(self.boards_to_format(ins))
        #         self.outputs.append(self.boards_to_format(out))

    # def boards_to_format(self, key):
    #     key = key[1:-1]
    #     return [int(c) for c in key.split(' ') if c in ['-1', '0', '1']]
        
    def __len__(self,):
        return self.len
    
    def __getitem__(self, idx):
        ins, outs = self.data[str(idx)]
        return ins, outs

In [7]:
train_dataset = TicTacDataset(path='./data/train.json')
val_dataset = TicTacDataset(path='./data/val.json')
test_dataset = TicTacDataset(path='./data/test.json')

In [8]:
from torch.utils.data import DataLoader

In [9]:
def collate(batch):
    # print(batch)
    ins = [torch.tensor(b[0])+1 for b in batch]
    # outs = [torch.tensor(np.average(np.array(b[1])+1, axis=0)) for b in batch]
    outs = [torch.tensor(b[1][0])+1 for b in batch]
    # print(outs)



    # max_outs = max([len(o) for o in outs])
    # padded_outs = []
    # for o in outs:
    #   pad_boxes = max_outs - len(o)
    #   padded_out = (o) + [[1,1,1,1,1,1,1,1,1]] * pad_boxes
    #   padded_outs.append(torch.tensor(padded_out, dtype=torch.float32)+1)
    return torch.stack(ins), torch.stack(outs)


In [31]:
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate)

In [11]:
out = next(iter(train_dataloader))

## Model Work

In [32]:
import torch
from torch import nn
import torch.nn.functional as F

torch.manual_seed(42)

# Set default device to CUDA if available, otherwise CPU
if torch.cuda.is_available():
    torch.set_default_device('cuda')
else:
    torch.set_default_device('cpu')

### Build Model

In [33]:
vocab_size = 3
num_embedding = 32

num_heads = 8

num_blocks = 4
block_size = 9

dropout = 0.0

In [34]:
class Head(nn.Module):
    """ Singular Head"""

    def __init__(self, num_embedding, head_size):
        super().__init__()
        self.key = nn.Linear(num_embedding, head_size, bias=False)
        self.query = nn.Linear(num_embedding, head_size, bias=False)
        self.value = nn.Linear(num_embedding, head_size, bias=False)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape

        k = self.key(x) # (B,T,C)
        q = self.query(x) # (B,T,C)

        # Computer attention
        # C**0.5 is to make the numbers smaller so softmax doesn't do weird things
        wei = q @ k.transpose(-2, -1) / C**0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        # 
        V = self.value(x)
        out = wei @ V # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [35]:
class MultiHeadAttention(nn.Module):
    """Multiple Attention Heads"""
    def __init__(self, num_heads, num_embedding, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(num_embedding, head_size) for i in range(num_heads)])
        self.project = nn.Linear(num_embedding, num_embedding)                  # Projection layer for gettting back into the residual pathway
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.project(out)
        out = self.dropout(out)
        return out

In [36]:
class FeedFoward(nn.Module):
    """Single Layer"""

    def __init__(self, num_embedding):
        super().__init__()

        self.m = nn.Sequential(nn.Linear(num_embedding, 4 * num_embedding), # 4* because they did it in the paper
                               nn.ReLU(),
                               nn.Linear(4 * num_embedding, num_embedding), # Projection layer for gettting back into the residual pathway
                               nn.Dropout(dropout)
        )
    
    def forward(self, x):
        return self.m(x)

In [37]:
class Block(nn.Module):
    """"""

    def __init__(self, num_embedding, num_head):
        super().__init__()

        head_size = num_embedding // num_head
        self.self_attn = MultiHeadAttention(num_head, num_embedding, head_size)
        self.feed_fwd = FeedFoward(num_embedding)

        self.lay_norm1 = nn.LayerNorm(num_embedding)
        self.lay_norm2 = nn.LayerNorm(num_embedding)

    def forward(self, x):
        x = x + self.self_attn(self.lay_norm1(x))
        x = x + self.feed_fwd(self.lay_norm2(x))
        return x


In [38]:
class TicTacToeModel(nn.Module):
    def __init__(self):
        super().__init__()

        # Embeddings
        self.token_embedding_table = nn.Embedding(vocab_size,num_embedding)
        self.position_embedding_table = nn.Embedding(block_size, num_embedding)

        # Attention
        self.attn_blocks = nn.Sequential(
            *[Block(num_embedding, num_head = 4) for i in range(num_blocks)],
            nn.LayerNorm(num_embedding),
        )

        self.lm_head = nn.Linear(num_embedding, vocab_size)

    def forward(self, inputs): #, labels=None):
        B, T = inputs.shape
        # print(inputs[0])
        # print(labels[0])

        token_embedding = self.token_embedding_table(inputs) # (B, T, C)
        position_embedding = self.position_embedding_table(torch.arange(T)) # (T, C)
        x = token_embedding + position_embedding # (B, T, C)

        x = self.attn_blocks(x)
        logits = self.lm_head(x) # (B, T, vocab_size)

        # if labels is None:
        #     loss = None
        # else:
        #     # print(logits.shape)
        #     # print(labels.shape)
            
        #     # loss = F.cross_entropy(logits, labels) # need to add 
        #     # loss = F.cross_entropy(logits.view(-1, logits.size(-1)), labels.view(-1), ignore_index=-1)
            
        return logits #, loss


## Train Model

In [39]:
from tqdm import tqdm

In [43]:
class ModelTrainer:
    def __init__(self, model,
                 train_dataloader,
                 val_dataloader, 
                 lr=1e-3,
                 epochs = 100,
                 device = None):
        self.model = model
        self.device = device if device is not None else ('cuda' if torch.cuda.is_available() else 'cpu')

        self.model.to(self.device)

        self.train_loader = train_dataloader
        self.val_loader = val_dataloader

        self.epochs = epochs
        self.lr = lr
        self.optimizer = torch.optim.AdamW(model.parameters(), lr=self.lr)
        self.criterion = nn.CrossEntropyLoss() 

    def train_one_epoch(self):
        self.model.train()
        running_loss = 0.0
        
        for inputs, labels in tqdm(self.train_loader, desc="Training", unit="batch"):
            inputs, labels = inputs.to(self.device), labels.to(self.device)
            
            self.optimizer.zero_grad()  # Zero the gradients
            logits = self.model(inputs)
            
            loss = self.criterion(logits.view(-1, logits.size(-1)), labels.view(-1))
            loss.backward()  # Backpropagate the loss
            self.optimizer.step()  # Update model parameters
            
            running_loss += loss.item()
        
        avg_train_loss = running_loss / len(self.train_loader)
        return avg_train_loss
    
    def evaluate(self):
        self.model.eval()
        correct = 0
        total = 0
        running_loss = 0.0
        
        with torch.no_grad():  # No need to track gradients during evaluation
            for inputs, labels in tqdm(self.val_loader, desc="Evaluating", unit="batch"):
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                logits = self.model(inputs)
                # print(f"logit shape: {logits.shape}")
                # print(f"labels shape: {labels.shape}")
                
                outputs = torch.argmax(logits, dim=2)
                # print(f"outputs shape: {outputs.shape}")
                
                loss = self.criterion(logits.view(-1, logits.size(-1)), labels.view(-1))
                running_loss += loss.item()
                
                # print(f"outputs:{outputs}")
                # print(labels.size(0))
                # print(outputs == labels)
                total += labels.size(0)
                correct += (torch.all(outputs == labels, dim=1)).sum().item()
                # input()
        
        avg_val_loss = running_loss / len(self.val_loader)
        accuracy = correct / total
        return avg_val_loss, accuracy
    
    def train(self):
        best_val_accuracy = 0.0
        
        for epoch in range(self.epochs):
            print(f"Epoch {epoch+1}/{self.epochs}")
            train_loss = self.train_one_epoch()
            val_loss, val_accuracy = self.evaluate()
            
            print(f"Train Loss: {train_loss:.4f}")
            print(f"Validation Loss: {val_loss:.4f}")
            print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
            
            # Save the best model based on validation accuracy
            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                print("Saving best model...")
                torch.save(self.model.state_dict(), "./models/best_model.pth")

    def load_best_model(self):
        self.model.load_state_dict(torch.load("./models/best_model.pth"))
        self.model.to(self.device)
        print("Best model loaded successfully.")

    def test(self, test_dataloader, model_path: str = None):
        if model_path is None:
            self.load_best_model()
        else:
            self.model.load_state_dict(torch.load(model_path))
            self.model.to(self.device)

        self.model.eval()
        correct = 0
        total = 0
        
        with torch.no_grad():  # No need to track gradients during evaluation
            for inputs, labels in tqdm(test_dataloader, desc="Testing", unit="batch"):
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                logits = self.model(inputs)
                
                
                outputs = torch.argmax(logits, dim=2)
                # print(f"outputs shape: {outputs.shape}")
                
                total += labels.size(0)
                correct += (torch.all(outputs == labels, dim=1)).sum().item()
                # input()
        
        accuracy = correct / total
        return accuracy
        

In [44]:
model = TicTacToeModel()
trainer = ModelTrainer(model, train_dataloader, val_dataloader, epochs=500)

In [42]:
trainer.train()

Epoch 1/500


Evaluating: 100%|██████████| 19/19 [00:00<00:00, 66.16batch/s]


Train Loss: 0.3727
Validation Loss: 0.2449
Validation Accuracy: 31.17%
Saving best model...
Epoch 2/500


Evaluating: 100%|██████████| 19/19 [00:00<00:00, 68.86batch/s]


Train Loss: 0.2041
Validation Loss: 0.1888
Validation Accuracy: 38.34%
Saving best model...
Epoch 3/500


...

Train Loss: 0.0000
Validation Loss: 0.1953
Validation Accuracy: 84.97%
Epoch 499/500


Evaluating: 100%|██████████| 19/19 [00:00<00:00, 72.53batch/s]


Train Loss: 0.0000
Validation Loss: 0.1957
Validation Accuracy: 84.88%
Epoch 500/500


Evaluating: 100%|██████████| 19/19 [00:00<00:00, 76.45batch/s]

Train Loss: 0.0000
Validation Loss: 0.2017
Validation Accuracy: 85.05%


In [45]:
trainer.test(test_dataloader)

Best model loaded successfully.


Testing: 100%|██████████| 10/10 [00:00<00:00, 61.76batch/s]


0.847972972972973